<a href="https://colab.research.google.com/github/stazam/ML-hackathon---genomic_benchmarks/blob/main/Introduction_script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#pip install genomic_benchmarks --upgrade
%%capture
!pip install genomic_benchmarks

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
import sys
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure


from keras.models import Sequential
from keras.layers import Dense, Dropout, Bidirectional, Activation, Flatten, MaxPooling1D, BatchNormalization, Conv1D
from keras import optimizers
from sklearn.model_selection import train_test_split
from pathlib import Path


from genomic_benchmarks.loc2seq import download_dataset
from genomic_benchmarks.data_check import is_downloaded, info, list_datasets

#Dataset - human_nontata_promoters 

- testovacia sada obsahuje chybajuce pozorovanie **N**, ktore sa nevyskytuje v treningovom sete, pretoho ho odstranim 

In [4]:
info('human_nontata_promoters', 0)

Dataset `human_nontata_promoters` has 2 classes: negative, positive.

All lenghts of genomic intervals equals 251.

Totally 36131 sequences have been found, 27097 for training and 9034 for testing.


,train,test
negative,12355,4119
positive,14742,4915


In [5]:
download_dataset("human_nontata_promoters", version=0)

Unzipping...Done.


PosixPath('/root/.genomic_benchmarks/human_nontata_promoters')

In [6]:
SEQ_PATH = Path.home() / '.genomic_benchmarks' / 'human_nontata_promoters'
CLASSES = [x.stem for x in (SEQ_PATH/'train').iterdir() if x.is_dir()]

train_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'train',
    batch_size=27097,
    class_names=CLASSES)

Found 27097 files belonging to 2 classes.


In [35]:
#tu mozno pridat PCA alebo nieco na zredukovanie dimenzie. Aj ked 251 je vpohode pre tento dataset

def preprocess_notNN(dat):

  dat_array = np.array(list(dat))

  labels = np.array(dat_array[0][1]).astype('float32')
  dataset = dat_array[0][0]

  ind_rem = []
  sequences_list = []
  for i,seq in enumerate(dataset):
    if 'N' not in str(seq):
      sequences_list.append(list(str(seq))[2:-1])
    else:
      ind_rem.append(i)  

  channels = {'A' : 0,'T' : 1,'C' : 2,'G' : 3}
  
  return np.array(pd.DataFrame(sequences_list).replace(channels)), np.delete(labels,ind_rem)

In [8]:
X_train, y_train = preprocess_notNN(train_dset)

In [9]:
SEQ_PATH = Path.home() / '.genomic_benchmarks' / 'human_nontata_promoters'
CLASSES = [x.stem for x in (SEQ_PATH/'test').iterdir() if x.is_dir()]

test_dset = tf.keras.preprocessing.text_dataset_from_directory(
    SEQ_PATH / 'test',
    batch_size=9034,
    class_names=CLASSES) 

Found 9034 files belonging to 2 classes.


In [10]:
X_test, y_test = preprocess_notNN(test_dset)

In [11]:
def preprocess_NN(dat):

  dat_array = np.array(list(dat))

  labels = np.array(dat_array[0][1]).astype('float32')
  dataset = dat_array[0][0]

  ind_rem = []
  sequences_list = []
  for i,seq in enumerate(dataset):
    if not 'N' in str(seq):
      sequences_list.append(list(str(seq))[2:-1])
    else:
      ind_rem.append(i)


  samples_size = len(sequences_list)
  sequence_size = min([len(x) for x in sequences_list])
  ohe = np.zeros((samples_size, sequence_size, 4))
  channels = {'A' : 0,'T' : 1,'C' : 2,'G' : 3}

  for index, sequence in enumerate(sequences_list):
    for pos, nucleotide in enumerate(sequence):
        ohe[index, pos, channels[nucleotide]] = 1
  
  return ohe, np.delete(labels,[ind_rem]), sequence_size

In [45]:
X_train, y_train, sequence_size = preprocess_NN(train_dset)
X_ev, y_ev, _ = preprocess_NN(test_dset)

In [13]:
print(X_ev.shape)
print(y_ev.shape)

(9033, 251, 4)
(9033,)


skusit ensemble s obycajnymi modeli a jednou CNN alebo potom este pridat jednu RNN a vyskusat to s nou

1. samostatne modey bez NN
2. modely s 1-nou NN - najprv CNN, potom RNN
3. modely s 2-ma NN - CNN, RNN
4. pouzit PCA na zredukovanie dimenzie

In [14]:
from google.colab import drive
#drive.mount('/content/drive',  force_remount=True)
drive.mount('/content/drive',  force_remount=True)


Mounted at /content/drive


In [15]:
#sys.path.append('/content/drive/MyDrive/CEITEC/Python_files')
#sys.path.append('/content/drive/MyDrive/Hockey_prediction/Python_script')
sys.path.append('/content/drive/MyDrive/ML_Hackathon/')

from help_functions import *

**Toto je moja najlepšia CNN architekrúra ktorou som dosiahol 86% (pár krát sa to prehuplo aj cez 87%).**

In [16]:
def CNN_win_model(sequence_size):
  model = Sequential([
        Conv1D(filters = 32, kernel_size=8, padding='same', activation = 'relu', input_shape=(sequence_size, 4)),
        Dropout(0.5),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(128, activation='relu'),
        Dense(64, activation='relu'),
        Dense(64, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

  model.summary()

  return model

**Toto je moja najlepšia CNN + Bi-LSTM architekrúra ktorou som dosiahol 86,5%.**


In [46]:
def CNN_LSTM_model(sequence_size):
  model = Sequential([
        Conv1D(filters = 32, kernel_size=8, padding='same', activation = 'relu', input_shape=(sequence_size, 4)),
        Bidirectional(keras.layers.LSTM(32, return_sequences=True)),
        Bidirectional(keras.layers.LSTM(16, return_sequences=True)),
        Dropout(0.5),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(16, activation='relu'),
        Dropout(0.5),
        Dense(1, activation='sigmoid')
    ])

  model.summary()

  return model



In [47]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, train_size = 0.8, random_state = 20)
model = CNN_model(sequence_size)

model.compile(
        optimizer='rmsprop',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

models_data = model.fit(X_train, y_train, batch_size=32, epochs=8, validation_data = (X_test, y_test))

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_3 (Conv1D)           (None, 251, 32)           1056      
                                                                 
 bidirectional_6 (Bidirectio  (None, 251, 64)          16640     
 nal)                                                            
                                                                 
 bidirectional_7 (Bidirectio  (None, 251, 32)          10368     
 nal)                                                            
                                                                 
 dropout_6 (Dropout)         (None, 251, 32)           0         
                                                                 
 flatten_3 (Flatten)         (None, 8032)              0         
                                                                 
 dense_7 (Dense)             (None, 128)              

In [48]:
metrics = model.evaluate(X_ev, y_ev, verbose=0)
print('model evaluation on unknown dataset [loss, accuracy]:', metrics)

model evaluation on unknown dataset [loss, accuracy]: [0.33043742179870605, 0.8612864017486572]


In [36]:
X_train, y_train = preprocess_notNN(train_dset)
X_test, y_test = preprocess_notNN(test_dset)

In [18]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences



In [ ]:
vocab_size = 10
embedding_dim = 40
max_length = 251
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"

In [18]:
def preprocess_notNN(dat):

  dat_array = np.array(list(dat))

  labels = np.array(dat_array[0][1]).astype('float32')
  dataset = dat_array[0][0]

  sequences_list = []
  for seq in dataset:
    if 'N' not in str(seq):
      sequences_list.append(list(str(seq))[2:-1])

  channels = {'A' : 0,'T' : 1,'C' : 2,'G' : 3}
  
  return np.array(pd.DataFrame(sequences_list).replace(channels)), labels

In [19]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(24, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

NameError: ignored